In [6]:
%pip install GitPython

  Obtaining dependency information for GitPython from https://files.pythonhosted.org/packages/e9/bd/cc3a402a6439c15c3d4294333e13042b915bbeab54edc457c723931fed3f/GitPython-3.1.43-py3-none-any.whl.metadata
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for gitdb<5,>=4.0.1 from https://files.pythonhosted.org/packages/fd/5b/8f0c4a5bb9fd491c277c21eff7ccae71b47d43c4446c9d0c6cff2fe8c2c4/gitdb-4.0.11-py3-none-any.whl.metadata
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Obtaining dependency information for smmap<6,>=3.0.1 from https://files.pythonhosted.org/packages/a7/a5/10f97f73544edcdef54409f1d839f6049a0d79df68adbc1ceb24d1aaca42/smmap-5.0.1-py3-none-any.whl.metadata
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
Using cached GitPython-3.1.43-py3-none-any.whl (207 kB)
Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
Using cached smmap-5.0.1-py3-none-any.whl (24 kB)

[notice] A new release of pip is ava

In [117]:
%pip install packaging


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [138]:
import git
from datetime import datetime, timezone
import requests
from datetime import datetime
import re
from packaging import version

In [139]:

def get_commit_timestamp(repo_path, commit_hash):
    try:
        repo = git.Repo(repo_path)
        commit = repo.commit(commit_hash)
        timestamp = commit.committed_date
        return timestamp
    except git.InvalidGitRepositoryError:
        print("Invalid Git repository.")
    except git.BadName:
        print("Invalid commit hash.")


In [144]:
def get_package_version(package_name, date):
    date = datetime.strptime(date, '%Y-%m-%d')
    url = f'https://pypi.org/pypi/{package_name}/json'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        releases = data['releases']
        new_releases = dict(sorted(releases.items(), key=lambda item: version.parse(item[0]), reverse=True))
        for version_, release_info in new_releases.items():
            for file_info in release_info:
                upload_time = file_info['upload_time']
                upload_time = datetime.strptime(upload_time, '%Y-%m-%dT%H:%M:%S')
                if upload_time <= date:
                    return version_

        return None
    else:
        raise Exception(f'Failed to fetch package information for {package_name}')

In [145]:
def get_dependencies_from_requirements(file_path):
    dependencies = []
    with open(file_path, 'r') as file:
        for line in file:
            # Use regex to match lines containing package names in requirements.txt format
            match = re.match(r'^\s*([a-zA-Z0-9_-]+)\s*(?:==.*|\n)?$', line)
            if match:
                dependencies.append(match.group(1))
    return dependencies

In [146]:
requirements_file = 'requirements.txt'
dependency_names = get_dependencies_from_requirements(requirements_file)

In [147]:
repo_path = "/Users/onkar/Documents/Code/GitRead/GitRead"
commit_hash = "3542d59e54882aa21419b233a1d99f6335224271" 
timestamp = get_commit_timestamp(repo_path, commit_hash)

if timestamp:
    timestamp = datetime.fromtimestamp(timestamp, tz=timezone.utc)
    formatted_timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S UTC')
    date_str = formatted_timestamp.split(' ')[0]
    print(f"Timestamp of commit {commit_hash}: {formatted_timestamp}")
    
    for package_name in dependency_names:
        version_ = get_package_version(package_name, date_str)
        if version_:
            print(f'The version of {package_name} at {date_str} was {version_}')
        else:
            print(f'No suitable version of {package_name} was found before {date_str}')

Timestamp of commit 3542d59e54882aa21419b233a1d99f6335224271: 2024-04-08 01:05:27 UTC
The version of numpy at 2024-04-08 was 2.0.0rc1
The version of pandas at 2024-04-08 was 2.2.1
The version of matplotlib at 2024-04-08 was 3.8.4
The version of scikit-learn at 2024-04-08 was 1.4.1.post1
The version of tensorflow at 2024-04-08 was 2.16.1
